In [480]:
import numpy as np
from numpy import random as rnd

In [481]:
# Use dispersity (between 1 and 3) to choose which system to return a radius from

def pickRadius(dispersity):
    if (dispersity == 1):
        return (1/((np.pi)**0.5))
    if (dispersity == 2):
        a = np.array([((3/(2*(np.pi)))**0.5), ((1/(2*(np.pi)))**0.5)])
        return rnd.choice(a , p = [0.5, 0.5])
    if (dispersity == 3):
        a = np.array([((15/(16*(np.pi)))**0.5), ((5/(4*(np.pi)))**0.5)])
        return rnd.choice(a , p = [0.8, 0.2])

In [482]:
#Class for creating a disc
# Cooridinates represent middle of discs
# choose 1-3 for dispersity

class config(): 
    def __init__(self, dispersity):
        self.radius = pickRadius(dispersity)
        self.x = (rnd.rand()) * 20
        self.y = (rnd.rand()) * 20
        return

In [483]:
# iterates attempts of N
#select dispersity scenario of 1-3

def addDiscs(N, dispersity, overlapConstant = 1.0):
    #defines new empty array
    situ = np.array([])
    
    for i in range(N):
        a = config(dispersity)
        #checks if disc is hitting sides
        insideBox = (
            ((a.x - a.radius > 0) and (a.x + a.radius < 20)) 
            and ((a.y - a.radius > 0) and (a.y + a.radius < 20))
        )
        
        # function to apply across array to check if new disc is hitting any exsisting discs
        def notHitting(a2):
            return (((((a.x - a2.x)**2) + ((a.y - a2.y)**2))**0.5) >
                (overlapConstant * (a.radius + a2.radius)))
        applyAll = np.vectorize(notHitting)
        
        #checking hitting any other disc, accounting for its the first disc to be placed
        if (situ.size == 0):
            notHitting = True
        else:
            hittingArray = applyAll(situ)
            notHitting = np.all(hittingArray)
        
        # appends disc to array if it meets conditions
        if (notHitting and insideBox):
            situ = np.append(situ, a)
            
        # Returns completed array after end of loop
    
    # Returns completed array after end of loop
    return situ
        

In [484]:
testArray = addDiscs(10,1)

print(testArray.size)
print(testArray)

5


In [520]:
# defines a class for a new random point on the square
class point(): 
    def __init__(self):
        #defines random x and y coordinates between 0 and 20
        self.x = (rnd.rand()) * 20
        self.y = (rnd.rand()) * 20
        return
    # a function that takes an array of discs and determines if the point is within any radius
    def inAnyCircle(self, discArray):
        for i in discArray: 
            inCircle = ((self.x - i.x)**2 + (self.y - i.y)**2)**0.5 < i.radius
            if (inCircle == True):
                return True
        return False
            
def packingFraction(discArray, NSample):
    validCount = 0
    for i in range(NSample):
        singlePoint = point()
        if (singlePoint.inAnyCircle(discArray) == True):
            validCount += 1
    return validCount / NSample

#need to both regenerate the discs and points every time, so below function is invalid

def averagedPackingFraction(NReps, discArray, NSample):
    fractions = np.array([])
    for i in range(NReps):
        fractions = np.append(fractions, (packingFraction(discArray, NSample)))
    return np.mean(fractions)


In [564]:
packingFraction(testArray, 200)

0.01

In [565]:
def combinedFunction(dispersity, overlapConstant, NParticles, NSample, NRep):
    fractions = np.array([])
    for i in range(NRep):
        discArray = addDiscs(NParticles, dispersity, overlapConstant)
        fractions = np.append(fractions, (packingFraction(discArray, NSample)))
    return np.mean(fractions)    
    

In [581]:
combinedFunction(3, 1, 1000, 1000, 100)

0.37979000000000007

In [587]:
import timeit
timeit.timeit(combinedFunction(3, 1, 100, 100, 10))

ValueError: stmt is neither a string nor callable